In [ ]:
# Creating live camera display

import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image]))

In [ ]:
from jetbot import Robot

robot = Robot()

In [ ]:
# Creating function to interprete the image and output predictions

import numpy as np
import tensorflow as tf
import time
import PIL
import io

category = ["Left", "Middle", "Right"]
prediction = ""

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="trace60.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

def update(change):
    img = change['new']
    
    # Line Tracing
    img = PIL.Image.open(io.BytesIO(image.value)).convert('L').resize((224, 224)) # .convert('L') changes image into grayscale
    img = np.array(img).astype(np.float32)
    img = img.reshape(1,224,224,1)
    
    interpreter.set_tensor(input_details[0]['index'], img)

    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    index_predicted = np.argmax(output_data[0])
    
    global prediction
    
    if category[index_predicted] != prediction:
        print(category[index_predicted])
        prediction = category[index_predicted]
    
    if index_predicted == 0:
        #robot.left(0.3)
        robot.left_motor.value = 0
        robot.right_motor.value = 0.3
    elif index_predicted == 1:
        #robot.forward(0.3)
        robot.left_motor.value = 0.3
        robot.right_motor.value = 0.3
    elif index_predicted == 2:
        #robot.right(0.3)
        robot.left_motor.value = 0.3
        robot.right_motor.value = 0
    
    time.sleep(0.001)

In [ ]:
update({'new': camera.value})  # we call the function once to intialize

In [ ]:
camera.observe(update, names='value') # this attaches the 'update' function to the 'value' traitlet of our camera

In [ ]:
camera.unobserve(update, names='value')
robot.stop()